<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Unconstrained-Model-Predictive-Control-(MPC)-on-Jupyter-Notebook" data-toc-modified-id="Unconstrained-Model-Predictive-Control-(MPC)-on-Jupyter-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Unconstrained Model Predictive Control (MPC) on Jupyter Notebook</a></span></li><li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-requisites</a></span></li><li><span><a href="#Background-Theory" data-toc-modified-id="Background-Theory-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Background Theory</a></span></li><li><span><a href="#Unconstrained-MPC" data-toc-modified-id="Unconstrained-MPC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Unconstrained MPC</a></span></li><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Prediction-matrices-(predicted-equality-constraint)" data-toc-modified-id="Prediction-matrices-(predicted-equality-constraint)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Prediction matrices (predicted equality constraint)</a></span></li><li><span><a href="#Rewriting-the-cost-function" data-toc-modified-id="Rewriting-the-cost-function-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Rewriting the cost function</a></span></li><li><span><a href="#Substituting-the-predicted-equality-constraint" data-toc-modified-id="Substituting-the-predicted-equality-constraint-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Substituting the predicted equality constraint</a></span></li><li><span><a href="#Cost-Function-in-QP-compact-form" data-toc-modified-id="Cost-Function-in-QP-compact-form-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Cost Function in QP compact form</a></span></li><li><span><a href="#Explicit-solution" data-toc-modified-id="Explicit-solution-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Explicit solution</a></span></li></ul></div>

# Unconstrained Model Predictive Control (MPC) on Jupyter Notebook

The following presents an unconstrained Linear Quadratic Model Predictive Control (LQ-MPC) simulation on Matlab-kernel for Jupyter Notebook. Also, the code files can be used in Matlab natively.

# Pre-requisites

- Knowledge in Control Systems theory.
- Convex quadratic optimization theory.
- Jupyter Notebook with Matlab-kernel.
- Matlab.


# Background Theory

MPC is an optimization-based control strategy that solves a Finite-Horizon Linear Quadratic (FH-LQ) problem subject to constraints. A model of predicted states based on the system dynamics is used to minimize an objective function that satisfies constraints given the actual state. The obtained solution is a sequence of open-loop controls from which only the first one is implemented on the system, inducing feedback. The rest of the sequence is discarded and the process is repeated for the next sampling time, this method is often called Receding-Horizon (RH) principle. Advantages of MPC against other control strategies are the handling of constraints, and non-linearities \cite{Rawlings2009,Rossiter2003}. 


In equation $\ref{test}$, we find the value of an
interesting integral:

\begin{equation}\tag{1}
  \int_0^\infty \frac{x^3}{e^x-1}\,dx = \frac{\pi^4}{15}  
\end{equation}

$$
\begin{align}
    g &= \int_a^b f(x)dx \tag{1} \\
    a &= b + c \tag{2}
\end{align}
$$
See (\ref{1}) and (\ref{2})

$$\label{eq:test} 
\begin{align}
M_y(t) &= \mathbb{E} \left( \exp \left[ t^\mathrm{T} (Ax + b) \right] \right) \\
&= \mathbb{E} \left( \exp \left[ t^\mathrm{T} A x \right] \cdot \exp \left[ t^\mathrm{T} b \right] \right) \\
&= \exp \left[ t^\mathrm{T} b \right] \cdot \mathbb{E} \left( \exp \left[ t^\mathrm{T} A x \right] \right) \\
&= \exp \left[ t^\mathrm{T} b \right] \cdot M_x(At) \; .
\end{align}
$$

$\eqref{eq:test}$

# Unconstrained MPC

Consider the following discrete-time LTI state-space system,
$$
\begin{equation}\label{eq:LTI}
\begin{aligned}
x(k+1) &= A~x(k)+B~u(k)\\
y(k) &= C~x(k),\quad k=0,1,2,\dots
\end{aligned}
\end{equation}
$$
where $x\in\mathbb{R}^n$, $u\in\mathbb{R}^m$, and $y\in\mathbb{R}^p$, are the states, input, and output vectors respectively. $A,~B,~C,$ are the state, input, and output matrices. $A,~B,~C$ are known, no uncertainties in the model, disturbances, and noise. $x(k)$ is available at each sampling time $k$.

# Problem Formulation

The objective is to regulate $x\rightarrow 0$ as $k\rightarrow \infty$ while minimizing the following cost function, 

\begin{equation}\label{eq:LQ-MPCu}
\begin{aligned}
J_N(x(k),\mathbf{u}(k)) &= \sum_{j=0}^{N-1} \left( \|x(k+j|k)\|^2_Q + \|u(k+j|k)\|^2_R \right) + \|x(k+N|k)\|^2_P\\
&= \sum_{j=0}^{N-1} (x^\intercal(k+j|k)~Q~x(k+j|k)+ u^\intercal(k+j|k)~R~u(k+j|k) ) + x^\intercal(k+N|k)~P~x(k+N|k) 
\end{aligned}
\end{equation}

\hspace{10em}subject to, 

\begin{equation}\label{eq:constraints_MPCu}
\begin{aligned}
x(k+1+j|k) &= A~x(k+j|k)+B~u(k+j|k) \\
x(k|k) &= x(k) 
\end{aligned}
\end{equation}

\hspace{10em} for $ j=0,1,2,\dots,N-1$\\

where $Q\in\mathbb{R}^{n\times n}$ and $R\in\mathbb{R}^{m\times m}$ are the state penalty, and input penalty matrices. $P\in\mathbb{R}^{n\times n}$ is the terminal cost matrix, $N \in\mathbb{N}$ is the horizon length, $k$ is the sampling time, and $(k+j|k)$ can be defined as the next $(k+j)$ value given $k$. 

With the value function defined as,
\begin{align}
J^*_N(x(k),\mathbf{u}(k)) &= \min_{u(k)}~ J_N(x(k),\mathbf{u}(k))
\end{align}
the optimal control sequence is,
\begin{equation}\label{eq:optimal_u}
\begin{aligned}
\mathbf{u}^*(k) &= \mathrm{arg}\min_{u(k)}~ J^*_N(x(k),\mathbf{u}(k))\\
&= \left\{ u^*(k|k),u^*(k+1|k),\dots,u^*(k+N-1|k) \right\}
\end{aligned}
\end{equation}

# Prediction matrices (predicted equality constraint)

Setting $x(k)=x(k|k)$, and $u(k)=u(k|k)$ and applying the model recursively,
$$
\begin{align*}
x(k+1|k) &= A~x(k)+B~u(k) \\
x(k+2|k) &= A~x(k+1|k)+B~u(k+1|k) \\
				 &= A^2~x(k)+AB~u(k|k)+B~u(k+1|k) \\
\vdots \hspace{2em}  &= \hspace{3em} \vdots \\
x(k+N|k) &= A~x(k+N-1|k)+B~u(k+N-1|k) \\
				 &= A^N~x(k)+A^{N-1}B~u(k|k)+\dots+B~u(k+N-1|k)
\end{align*}
$$

stacking,

$$
\begin{aligned}
{\underbrace{
	\begin{bmatrix}
	x(k+1|k) \\
	x(k+2|k) \\
	\vdots \\
	x(k+N|k)
	\end{bmatrix}}_{\mathbf{x}(k)}
}
=
{\underbrace{
	\begin{bmatrix}
	A \\
	A^2 \\
	\vdots \\
	A^N
	\end{bmatrix}}_{F}
}
+
{\underbrace{
	\begin{bmatrix}
		B        & 0        & \dots  & 0      \\
		AB       & B        & \dots  & 0      \\
		\vdots   & \vdots   & \ddots & \vdots \\
		A^{N-1}B & A^{N-2}B & \dots  & B
	\end{bmatrix}}_{G}
}
{\underbrace{
	\begin{bmatrix}
	u(k|k) \\
	u(k+1|k) \\
	\vdots \\
	u(k+N-1|k)
	\end{bmatrix}}_{\mathbf{u}(k)}
}
\end{aligned}
$$
the predicted equality constraint is,
$$
\begin{equation}
\mathbf{x}(k) = F~x(k)+G~\mathbf{u}(k)
\end{equation}
$$
where $\mathbf{x}(k)$, and $\mathbf{u}(k)$ are the state, and input predictions over all steps $j=0,1,2,\dots,N$.




# Rewriting the cost function

Rewriting the cost function as,
$$
\begin{aligned}
x^\intercal(k|k)~Q~x(k|k)
&+
\begin{bmatrix}
	x(k+1|k) \\
	x(k+2|k) \\
	\vdots\\
	x(k+N|k) \\
\end{bmatrix}^\intercal
{\underbrace{
	\begin{bmatrix}
		Q      & 0      & \dots  & 0      \\
		0      & Q      & \ddots & \vdots \\
		\vdots & \ddots & Q      & 0      \\
		0      & \dots  & 0      & P
	\end{bmatrix}}_{\tilde{Q}}
}
\begin{bmatrix}
	x(k+1|k) \\
	x(k+2|k) \\
	\vdots\\
	x(k+N|k) \\
\end{bmatrix}
\\
&+
\begin{bmatrix}
	u(k|k) \\
	u(k+1|k) \\
	\vdots \\
	u(k+N-1|k)
\end{bmatrix}^\intercal
{\underbrace{
	\begin{bmatrix}
	R      & 0      & \dots  & 0      \\
	0      & R      & \ddots & \vdots \\
	\vdots & \ddots & R      & 0      \\
	0      & \dots  & 0      & R
		\end{bmatrix}}_{\tilde{R}}
}
\begin{bmatrix}
	u(k|k) \\
	u(k+1|k) \\
	\vdots \\
	u(k+N-1|k)
\end{bmatrix}
\end{aligned}
$$

Therefore, with $x(k|k)=x(k)$, now the problem is to minimize,
$$
\begin{align}
J_N(x(k),\mathbf{u}(k)) &=  x^\intercal(k)~Q~x(k) + \mathbf{x}^\intercal(k)~\tilde{Q}~\mathbf{x}(k) + \mathbf{u}^\intercal(k)~\tilde{R}~\mathbf{u}(k)\\ 
\text{subject to,} \hspace{2em} \\
\mathbf{x}(k) &= F~x(k)+G~\mathbf{u}(k)
\end{align}
$$

# Substituting the predicted equality constraint

$$
\begin{align*}
J_N(x(k),\mathbf{u}(k)) &=  x^\intercal(k) Q x(k) + ( F x(k)+G \mathbf{u}(k) )^\intercal \tilde{Q} ( F x(k)+G \mathbf{u}(k) ) + \mathbf{u}^\intercal(k)~\tilde{R} \mathbf{u}(k) \\
\text{omitting $(k)$ only for the algebraic operations,}\\
&= x^\intercal Q x + [(F x)^\intercal+(G \mathbf{u})^\intercal] \tilde{Q} (F x+G \mathbf{u}) + \mathbf{u}^\intercal \tilde{R} \mathbf{u} \\
&= x^\intercal Q x + (F x)^\intercal \tilde{Q} F x + (F x)^\intercal \tilde{Q} G \mathbf{u} + (G \mathbf{u})^\intercal \tilde{Q} F x + (G \mathbf{u})^\intercal \tilde{Q} G \mathbf{u} + \mathbf{u}^\intercal~\tilde{R} \mathbf{u}\\
&= \underbrace{x^\intercal Q x + x^\intercal F^\intercal \tilde{Q} F x} + \underbrace{x^\intercal F^\intercal \tilde{Q} G \mathbf{u} + \mathbf{u}^\intercal G^\intercal \tilde{Q} F x} + \underbrace{\mathbf{u}^\intercal G^\intercal \tilde{Q} G \mathbf{u} + \mathbf{u}^\intercal \tilde{R} \mathbf{u}}\\
&= x^\intercal(Q+F^\intercal \tilde{Q} F)x \quad \!\!+ \quad 2 \mathbf{u}^\intercal G^\intercal \tilde{Q} F x \qquad\quad+ \mathbf{u}^\intercal (G^\intercal \tilde{Q} G+\tilde{R})\mathbf{u}\\
&= x^\intercal(Q+F^\intercal \tilde{Q} F)x + (2 G^\intercal \tilde{Q} F x)^\intercal \mathbf{u} + \mathbf{u}^\intercal (G^\intercal \tilde{Q} G+\tilde{R})\mathbf{u}\\
&= x^\intercal\underbrace{(Q+F^\intercal \tilde{Q} F)}_M x + (\underbrace{2 G^\intercal \tilde{Q} F}_L x)^\intercal \mathbf{u} + \frac{1}{2}\left[\mathbf{u}^\intercal \underbrace{2(G^\intercal \tilde{Q} G+\tilde{R})}_H \mathbf{u}\right]\\
&= \frac{1}{2}\mathbf{u}(k)^\intercal~H~\mathbf{u}(k)+\underbrace{(L~x(k))^\intercal}_{c^\intercal}~\mathbf{u}(k)+\underbrace{x(k)^\intercal~M~x(k)}_\alpha
\end{align*}
$$

# Cost Function in QP compact form

$$
\begin{align}
J_N(x(k),\mathbf{u}(k)) &= \frac{1}{2} \mathbf{u}(k)^\intercal~H~\mathbf{u}(k)+c^\intercal~\mathbf{u}(k)+\alpha \\
\text{where,} \hspace{3em} \\
H&=2(G^\intercal\tilde{Q}G+\tilde{R})\\
c&=L~x(k) \\
L&=2G^\intercal\tilde{Q}F\\
\alpha&=x^\intercal(k)~M~x(k) \\
M&=Q+F^\intercal\tilde{Q}F
\end{align}
$$

# Explicit solution

Minimizing with the gradient and solving for $\mathbf{u}^*(k)$,

$$
\begin{align}
J^*_N(x(k),\mathbf{u}(k)) &= \min_{\mathbf{u}(k)}~\frac{1}{2} \mathbf{u}(k)^\intercal~H~\mathbf{u}(k)+c^\intercal~\mathbf{u}+\alpha \\ 
\nabla_\mathbf{u}~J^*_N(x(k),\mathbf{u}(k)) &= 0 \\
H~\mathbf{u}^*(k)+c &= 0 \\
\mathbf{u}^*(k) &= -H^{-1}~c \\
\text{the optimal solution results in,} \hspace{5em} \\
\mathbf{u}^*(k) &= -H^{-1}~L~x(k)
\end{align}
$$
where that $Q\succeq,~P\succeq 0$ are positive semidefinite convex, and $R\succ 0$ is definite convex, such that $H\succ 0$. The optimal solution $\mathbf{u}^*(k)$ for any $x(k)$ is unique if $H\succ 0$ is invertible.

Applying the RH principle means that only the \textbf{first} control input of the vector \eqref{eq:opt_mpc_u} is applied to the real system. That is,
$$
\begin{align}
\begin{bmatrix}
	u^*(k) \\
	u^*(k+1) \\
	\vdots \\
	u^*(k+N-1)
\end{bmatrix}
&=
\begin{bmatrix}
	I      & 0      & 0      & \dots  & 0      \\
	0      & I      & 0      & \dots  & 0      \\
	\vdots & \vdots & \vdots & \ddots & \vdots \\
	0      & 0      & 0      & \dots  & I
\end{bmatrix}
(-H^{-1}~L)x(k)
\end{align}
$$

$$
\begin{align}
u^*(k) &= K_N~x(k)\\
K_N &= [I \quad 0 \quad 0 \quad \dots \quad 0](-H^{-1}~L)
\end{align}
$$
Notice that $u^*(k)$ is an explicit linear time-invariant control law because $H$ and $L$ do not depend on $x(k)$.


\begin{align}\label{eq:test2}
u^*(k) &= K_N~x(k)\\
K_N &= [I \quad 0 \quad 0 \quad \dots \quad 0](-H^{-1}~L)
\end{align}

Notice that $u^*(k)$ is an explicit linear time-invariant control law because $H$ and $L$ do not depend on $x(k)$. \eqref{eq:test2}